In [1]:
import os
import bcolz
import json

import tensorflow as tf
import numpy as np
import pandas as ps

%reload_ext autoreload
%autoreload 2

%aimport dataset_interval_reader

from shared_examples_queue import SharedExamplesQueue

### Write some dummy test data

In [20]:
NUM_SEQ_CHARS = 4
SEQ_LEN_CHR = int(1e5)
NUM_CHRS = 5
NUM_DATASETS = 3

NUM_INTERVALS = 500
INTERVAL_LENGTH = 1000
NUM_TASKS = 6

DATA_DIR = 'test-data'
FA_DIRS = [os.path.join(DATA_DIR, 'seq-{}'.format(i)) for i in range(NUM_DATASETS)]
BW_DIRS = [os.path.join(DATA_DIR, 'bgw-{}'.format(i)) for i in range(NUM_DATASETS)]

INTERVALS_FILE = os.path.join(DATA_DIR, 'intervals_file.json')
INPUTS_FILE = os.path.join(DATA_DIR, 'inputs_file.json')

TASK_NAMES = ['task-name-{}'.format(i) for i in range(NUM_TASKS)]
DATASET_NAMES = ['dataset-name-{}'.format(i) for i in range(NUM_DATASETS)]

BLOSC_CPARAMS = bcolz.cparams(clevel=5, shuffle=bcolz.SHUFFLE, cname='lz4')

if not os.path.isdir(DATA_DIR):
    os.mkdir(DATA_DIR)

for FA_DIR in FA_DIRS:
    if not os.path.isdir(FA_DIR):
        os.mkdir(FA_DIR)

for BW_DIR in BW_DIRS:
    if not os.path.isdir(BW_DIR):
        os.mkdir(BW_DIR)

def random_fasta_seq():
    seq_idxs = np.random.randint(0, NUM_SEQ_CHARS, SEQ_LEN_CHR)
    seq_arr = np.zeros((NUM_SEQ_CHARS, SEQ_LEN_CHR))
    seq_arr[seq_idxs, np.arange(SEQ_LEN_CHR, dtype=int)] = 1
    return seq_arr

def random_bw_data():
    # Just use low-frequency wave function for now
    bw_data = np.sin(np.arange(SEQ_LEN_CHR) / 1e-3)
    return bw_data

def random_labels():
    # Just random labels for now, as ints
    labels = np.random.randint(0, 3, size=(NUM_INTERVALS, NUM_TASKS))
    return labels

def random_intervals():
    interval_starts = np.random.randint(0, SEQ_LEN_CHR - INTERVAL_LENGTH, size=NUM_INTERVALS)
    interval_ends = interval_starts + INTERVAL_LENGTH
    interval_chrs = np.random.randint(0, NUM_CHRS, size=NUM_INTERVALS)
    interval_chrs = np.array(list(map(lambda x: 'chr{}'.format(x), interval_chrs)))
    intervals = ps.DataFrame([interval_chrs, interval_starts, interval_ends]).T
    return intervals
    

seq_arrs = {'chr{}'.format(i): random_fasta_seq() for i in range(NUM_CHRS)}
bw_arrs = {'chr{}'.format(i): random_bw_data() for i in range(NUM_CHRS)}

def dump_to_disk(chr_key, arr, base_dir):
    target_fname = os.path.join(base_dir, chr_key)
    c_arr = bcolz.carray(arr, cparams=BLOSC_CPARAMS, rootdir=target_fname, mode='w')
    c_arr.flush()

def write_metadata(base_dir):
    # Check the first file to get the shape
    arr_shape = bcolz.carray(rootdir=os.path.join(base_dir, 'chr0'), mode='r').shape
    chr_shapes = {'chr{}'.format(i): arr_shape for i in range(NUM_CHRS)}
    metadata = {'type': 'array_bcolz', 'file_shapes': chr_shapes}
    with open(os.path.join(base_dir, 'metadata.json'), 'w') as fp:
        json.dump(metadata, fp)

for FA_DIR in FA_DIRS:
    for chr_key, arr in seq_arrs.items():
        dump_to_disk(chr_key, arr, FA_DIR)
    write_metadata(FA_DIR)

for BW_DIR in BW_DIRS:
    for chr_key, arr in bw_arrs.items():
        dump_to_disk(chr_key, arr, BW_DIR)
    write_metadata(BW_DIR)
    
intervals_file_dict = {'task_names': TASK_NAMES}
inputs_file_dict = {}

for dataset_idx, dataset_name in enumerate(DATASET_NAMES):
    labels_file = os.path.join(DATA_DIR, 'labels{}.npy'.format(dataset_idx))
    intervals_file = os.path.join(DATA_DIR, 'intervals{}.bed'.format(dataset_idx))
    
    labels = random_labels()
    np.save(labels_file, labels)
    
    intervals = random_intervals()
    intervals.to_csv(intervals_file, sep='\t', header=False, index=False)
    
    intervals_file_dict[dataset_name] = {'regions': intervals_file, 'labels': labels_file}
    inputs_file_dict[dataset_name] = {'dnase_data_dir': BW_DIRS[dataset_idx], 'genome_data_dir': FA_DIRS[dataset_idx]}

with open(INTERVALS_FILE, 'w') as fp:
    json.dump(intervals_file_dict, fp)

with open(INPUTS_FILE, 'w') as fp:
    json.dump(inputs_file_dict, fp)


### Set up the readers

The data we want to read is in `test-data/`.

We just need to use `dataset_interval_reader.get_readers` to create readers for all the datasets

In [3]:
readers, task_names = dataset_interval_reader.get_readers_and_tasks(INPUTS_FILE, INTERVALS_FILE)

In [4]:
s = tf.InteractiveSession()

In [5]:
s.run(tf.global_variables_initializer())
s.run(tf.local_variables_initializer())

# Note that you *must* start queue runners before fetching any of the dequeues.
queue_runner_threads = tf.train.start_queue_runners(s)

In [6]:
# Note that readers is a dictionary of `examples_queue`s
readers

{u'dataset-name-0': <tensorflow.python.ops.data_flow_ops.FIFOQueue at 0x1147eb790>,
 u'dataset-name-1': <tensorflow.python.ops.data_flow_ops.FIFOQueue at 0x1147eb990>,
 u'dataset-name-2': <tensorflow.python.ops.data_flow_ops.FIFOQueue at 0x114d7ea10>}

In [7]:
# We can fetch from any of the readers like this
s.run(readers['dataset-name-0'].dequeue())

{'data/dnase_data_dir': array([ 0.53336602, -0.7812708 , -1.39517558, -0.77103299,  0.54488111,
         1.40082371,  1.04763794, -0.2055531 , -1.26190436, -1.1968528 ,
        -0.06733441,  1.13804913,  1.36429572,  0.4133848 , -0.88240653,
        -1.38894749, -0.66289222,  0.66028523,  1.42248476,  0.95659727,
        -0.32961288, -1.31040084, -1.12733984,  0.05934735,  1.2110225 ,
         1.31969142,  0.29024231, -0.97630769, -1.3714211 , -0.54927814,
         0.77054733,  1.43288863,  0.85803723, -0.45087314, -1.34822929,
        -1.04862738,  0.18570818,  1.27443552,  1.26465476,  0.1649265 ,
        -1.06222105, -1.34273708, -0.43110216,  0.87478262,  1.43195248,
         0.75274867, -0.56836092, -1.37508595, -0.96134698,  0.31073433,
         1.32777917,  1.1996274 ,  0.03844271, -1.13945735, -1.30312538,
        -0.30931234,  0.97215509,  1.41968286,  0.64157611, -0.68113375,
        -1.39075565, -0.86619872,  0.43342274,  1.37062573,  1.12513101,
        -0.08819428, -1.2073

In [8]:
# Or fetch a batch like this
_ = s.run(readers['dataset-name-0'].dequeue_many(128))

In [9]:
# We can also check how fast it is,
# but note that this is slower because the full result is returned to python

def fetch():
    _ = s.run(readers['dataset-name-1'].dequeue_many(128))

%timeit fetch()

100 loops, best of 3: 28.7 ms per loop


### Set up a parallel examples queue
This takes the set of readers as input, and provides a shuffled buffer of examples from all datasets in a defined batch size

In [10]:
shared_queue = SharedExamplesQueue(readers)

In [11]:
s.run(tf.global_variables_initializer())
s.run(tf.local_variables_initializer())

# Note that you *must* start queue runners before fetching.
queue_runner_threads = tf.train.start_queue_runners(s)

In [15]:
# The shared queue provides outputs shuffled across all the datasets:

shared_queue.outputs

{'data/dnase_data_dir': <tf.Tensor 'shared-examples-queue/shuffle_batch:0' shape=(128, 100) dtype=float32>,
 'data/genome_data_dir': <tf.Tensor 'shared-examples-queue/shuffle_batch:1' shape=(128, 4, 100) dtype=float32>,
 'dataset': <tf.Tensor 'shared-examples-queue/shuffle_batch:2' shape=(128,) dtype=string>,
 'dataset-index': <tf.Tensor 'shared-examples-queue/shuffle_batch:3' shape=(128,) dtype=int32>,
 'intervals/chrom': <tf.Tensor 'shared-examples-queue/shuffle_batch:4' shape=(128,) dtype=string>,
 'intervals/end': <tf.Tensor 'shared-examples-queue/shuffle_batch:5' shape=(128,) dtype=int64>,
 'intervals/start': <tf.Tensor 'shared-examples-queue/shuffle_batch:6' shape=(128,) dtype=int64>,
 'labels': <tf.Tensor 'shared-examples-queue/shuffle_batch:7' shape=(128, 6) dtype=int64>}

In [18]:
# It also provides the dataset (and the `dataest-index`) that each example came from

# This is intended to make it easy to analyze performance by dataset

print(shared_queue.outputs['dataset'].eval()[:10])

print(shared_queue.outputs['dataset-index'].eval()[:10])

['dataset-name-2' 'dataset-name-1' 'dataset-name-0' 'dataset-name-2'
 'dataset-name-1' 'dataset-name-1' 'dataset-name-0' 'dataset-name-2'
 'dataset-name-2' 'dataset-name-0']
[2 1 0 2 2 0 2 2 2 0]
